In [1]:
import pdfplumber
import pandas as pd

# Create df from table on first page to act as the first df:
pdf_file = "purchaser_details.pdf"
pdf = pdfplumber.open(pdf_file)
pages = pdf.pages
tbl = pages[0].extract_table()
original_df = pd.DataFrame(tbl)

# Use the first row as the header:
original_df.columns = original_df.iloc[0]
original_df = original_df[1:]  # Remove the first row, as it's now the header

# Append data from remaining tables/pages:
tables = []
with pdfplumber.open(pdf_file) as pdf:
    for i, pg in enumerate(pdf.pages[1:], start=1):  # Start from the second page (index 1)
        tbl = pg.extract_table()
        if tbl:
            df = pd.DataFrame(tbl)
            df.columns = original_df.columns  # Set the columns to match the header
            tables.append(df)

if tables:
    # Concatenate all dataframes in the tables list along axis=0
    original_df = pd.concat([original_df] + tables, ignore_index=True)

print(original_df)


C:\Users\mdhru\AppData\Local\Temp\ipykernel_10784\23967222.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


0     Date of Purchase                  Purchaser Name Denomination
0          12/Apr/2019             A B C INDIA LIMITED     1,00,000
1          12/Apr/2019             A B C INDIA LIMITED     1,00,000
2          12/Apr/2019             A B C INDIA LIMITED    10,00,000
3          12/Apr/2019             A B C INDIA LIMITED    10,00,000
4          12/Apr/2019             A B C INDIA LIMITED     1,00,000
...                ...                             ...          ...
18866      11/Jan/2024  WESTWELL GASES PRIVATE LIMITED  1,00,00,000
18867      11/Jan/2024  WESTWELL GASES PRIVATE LIMITED  1,00,00,000
18868      11/Jan/2024  WESTWELL GASES PRIVATE LIMITED  1,00,00,000
18869      11/Jan/2024  WESTWELL GASES PRIVATE LIMITED  1,00,00,000
18870      11/Jan/2024  WESTWELL GASES PRIVATE LIMITED  1,00,00,000

[18871 rows x 3 columns]


In [2]:
# Save the DataFrame as a CSV file
original_df.to_csv('purchaser_details_output.csv', index=False)


In [6]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('purchaser_details_output.csv')

# Display basic statistics and information about the DataFrame
print(df.describe())


       Date of Purchase                       Purchaser Name Denomination
count             18871                                18871        18871
unique              146                                 1316            5
top         05/Jan/2022  FUTURE GAMING AND HOTEL SERVICES PR  1,00,00,000
freq                570                                 1208        11671


In [7]:
import pandas as pd

# Convert "Denomination" column to numeric type after removing commas
df['Denomination'] = df['Denomination'].str.replace(',', '').astype(float)

# Group by "Name of the Political Party" and find the sum of "Denomination"
grouped_df = df.groupby('Purchaser Name')['Denomination'].sum().reset_index()

# Sort the grouped DataFrame in descending order based on "Denomination"
sorted_df = grouped_df.sort_values(by='Denomination', ascending=False)

# Convert "Denomination" column to numeric type after removing commas (assuming commas were already removed)
sorted_df['Denomination'] = sorted_df['Denomination'].astype(float)

# Divide "Denomination" values by 10 million to represent in crores
sorted_df['Denomination in Crores'] = sorted_df['Denomination'] / 10**7

# Display the top 20 rows of the DataFrame with Denomination in crores
print(sorted_df[['Purchaser Name', 'Denomination in Crores']].head(20))



                                      Purchaser Name  Denomination in Crores
354              FUTURE GAMING AND HOTEL SERVICES PR                 1208.00
652   MEGHA ENGINEERING AND INFRASTRUCTURES LI MITED                  821.00
866                    QWIKSUPPLYCHAINPRIVATELIMITED                  410.00
417                            HALDIA ENERGY LIMITED                  377.00
1248                                 VEDANTA LIMITED                  375.65
334                        ESSEL MINING AND INDS LTD                  224.50
1299  WESTERN UP POWER TRANSMISSION COMPANY LI MITED                  220.00
556                  KEVENTER FOODPARK INFRA LIMITED                  195.00
619                                    MADANLAL LTD.                  185.50
174                            BHARTI AIRTEL LIMITED                  183.00
1306               YASHODA SUPER SPECIALITY HOSPITAL                  162.00
1221             UTKAL ALUMINA INTERNATIONAL LIMITED                  135.30